# Explore Sequence Diversity

This notebook is for exploring protein sequence diversity in a set of CAZymes, e.g. a CAZy family.

Use `cazy_webscraper` to retrieve the protein sequences for your dataset of interest, and extract the protein sequences to a FASTA file. Use the provided `run_blastp.sh` or `run_diamond.sh` bash scripts to use BLASTP+ or DIAMOND, respectively, to perform an all-vs-all sequence comparison across your dataset of interest. 

This notebook takes as input the output from BLASTP+/DIAMOND and visualises the data. 

Feel free to use this notebook as a template to perform further analyses.

## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys
import time

import mechanicalsoup

from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError, MissingSchema
from tqdm.notebook import tqdm
from urllib3.exceptions import HTTPError, RequestError

## Constants

Define proteins of interest, e.g. proteins to be explored in the lab. These will be highlighed on the resulting clustermaps and heatmaps.

The `dict` uses the group name (e.g. a CAZy family) as the key, and is valued by a list of the NCBI protein version accessions.

In [ ]:
CANDIDATES = {
    'grp_name': ['protein_acc']
}

In [ ]:
# define the colour palettes for annotating proteins
PALETTE = sns.color_palette(['#425df5', '#eb8913', '#19bfb4', '#db0d4e', '#15ab62', '#ffffff'])
PALETTE_DICT = {
    'cand': '#425df5',  # candidates
    'struct': '#eb8913',  # protein with structures in RCSB PDB
    'structCand': '#19bfb4',  # candidates with structures in RCSB PDB
    'func': '#db0d4e',  # candidates listed as 'characterised' in CAZy
    'funcCand': '#15ab62',  # proteins listed as 'characterised' in CAZy
    'nothing': '#ffffff',  # nothing to note about this protein
}

## Identify proteins of interest

The clustermaps and heatmaps generated to visualise the sequence diversity across the CAZyme database, can be annotated to identify proteins of interest (e.g. candidates, see above), functionally characterised proteins (proteins listed as 'characterised' in CAZy) and structurally characterised proteins (listed under 'structures' in CAZy and also be retrieved from UniProt using `cazy_webscraper`.

In [ ]:
# Run this code block before retrieving data from CAZy

def browser_decorator(func):
    """Decorator to re-invoke the wrapped function up to 10 times."""

    def wrapper(*args, **kwargs):
        tries, success, err = 0, False, None

        while not success and (tries < kwargs['max_tries']):
            try:
                response = func(*args, **kwargs)
            except (
                ConnectionError,
                HTTPError,
                OSError,
                MissingSchema,
                RequestError,
            ) as err_message:
                if (tries < kwargs['max_tries']):
                    print(
                        f"Failed to connect to CAZy on try {tries}/{kwargs['max_tries']}.\n"
                        f"Error: {err_message}"
                        "Retrying connection to CAZy in 10s"
                    )
                success = False
                response = None
                err = err_message
            if response is not None:  # response was successful
                success = True
            # if response from webpage was not successful
            tries += 1
            time.sleep(10)
            
        if (not success) or (response is None):
            print(f"Failed to connect to CAZy.\nError: {err}")
            return None, err
        else:
            return response, None

    return wrapper


@browser_decorator
def get_page(url, **kwargs):
    """Create browser and use browser to retrieve page for given URL.

    :param url: str, url to webpage
    :param args: cmd-line args parser
    :kwargs max_tries: max number of times connection to CAZy can be attempted

    Return browser response object (the page).
    """
    # create browser object
    browser = mechanicalsoup.Browser()
    # create response object
    page = browser.get(url, timeout=45)
    page = page.soup

    return page


def get_all_accessions(bs_element_lst):
    """Retrieve all accessions listed in a cell from a CAZyme table.

    :param bs_element_list: list of BeautifulSoup element from cell in HTML table

    Return list of accessions."""
    accessions = []

    for bs_element in bs_element_lst:
        try:
            if bs_element.name == "a":  # Hyperlinked, extract accession and add to primary
                accessions.append(bs_element.text)
            elif bs_element.strip() != "":  # There is text in element
                accessions.append(bs_element)
        except TypeError:
            pass

    return accessions


def get_cazy_prots(url, col_index, file_path=None):
    """Get the NCBI protein accessions for proteins in the structure or characterised tables.
    
    :param url: str, URL to the CAZy family structure tabel
    :param col_index: int, 3 = structure, 4 = characterised
    
    Return list of NCBI protein accessions
    """
    if file_path is None:
        page, error_mss = get_page(
            url,
            max_tries=100
        )
        if page is None:
            print('Did not retrieve page')
            print(error_mss)
            return
    else:
        with open(file_path, "r") as fp:
            page = BeautifulSoup(fp, features="lxml")
    
    cazyme_table = page.select('table')[1]

    gbk_bs_elements = []

    for row in cazyme_table.select("tr"):
        try:
            if (row.attrs["class"] == ['royaume']) and (row.text.strip() != 'Top'):
                continue
        except KeyError:
            pass

        try:
            if (row.attrs["id"] == 'line_titre'):
                continue
        except KeyError:
            pass

        try:
            gbk_bs_elements += [_ for _ in row.select("td")[col_index].contents if getattr(_, "name", None) != "br"]
        except IndexError:
            pass

    ncbi_accessions = get_all_accessions(gbk_bs_elements)
    
    ncbi_accessions = list(set(ncbi_accessions))
    
    return ncbi_accessions

## Get 'characterised' proteins from CAZy

To retrieve the protein listed as 'characterised' in CAZy, for each CAZy family of interest call `get_cazy_prots` and provide:
* The URL to the CAZy family characterised page, e.g. 'http://www.cazy.org/PL1_characterized.html'
* Provide the index of the html table column containing the NCBI protein accessions, this is index 4 for the characterised table

Optionally, if you have saved the HTML page from CAZy, you can provide the path to the HTML file and data can be retrieved directly from this file, e.g. file_path="pages/CAZy - PL1.html". This may be necessary when retrieving data from many CAZy pages, because CAZy will close the connection if too many frequent calls to CAZy are made.

`get_cazy_prots()` returns a list of NCBI protein version accessions. 

Store this list in the `functioned_prots` dict, keyed by the name of the CAZy family and valued by the list of NCBI protein version accessions.

In [ ]:
functioned_prots = {}
functioned_prots['PL1'] = get_cazy_prots('http://www.cazy.org/PL1_characterized.html', 4, file_path="pages/CAZy - PL1.html")
functioned_prots

## Get structurally characterised proteins

As with the 'characterised' proteins, proteins in the CAZy structure table for each CAZy family can be retrieved directly from CAZy. See the 'Get 'characterised' proteins from CAZy' above, but use column index 3.

In [ ]:
structured_prots = {}
structured_prots['PL1'] = get_cazy_prots('http://www.cazy.org/PL1_structure.html', 3)
structured_prots

Not all structurally characterised members of a CAZy family are listed in CAZy. The structurally characterised members of a CAZy family can also be retrieved from UniProt using `cazy_webscraper`.

Specifically, use `cw_get_uniprot_data` to retrieve the RCSB PDB ids for proteins in your CAZy families of interest:
```
cw_get_uniprot_data <path to local cazyme db> --pdb
```
Use `cw_query_database` to generate a `JSON` file keyed by the NCBI protein version acession and valued by a dict. This `JSON` file can be parsed and added to the `structured_prots` dict.
```
cw_query_database <path to local cazyme db>
```

We recommend using data retrieved from CAZy **and** UniProt because there are proteins listed as structurally characterised in CAZy that are not labelled as such in UniProt, and vice versa. Therefore, using data from both databases will provide the most comprehensive representation of the structural characterisation of the family.

In [ ]:
def get_uniprot_structure_prots():
    """Identify proteins with a least one PDB protein structure ID in UniProt
    
    :param:
    
    Return list of NCBI protein version accessions
    """

In [ ]:
structured_prots['PL1'].append(get_uniprot_structure_prots())
structured_prots

## Load data

Load the BLASTP/DIAMOND data into a `pandas` dataframe.

CAZy can contain groups of identical proteins. Optionally, these groups can be removed and one representative from group retained, to remove redundant sequences from downstream analyses. When removing redundant proteins, all proteins labelled as candidates, structurally characterised and/or functionally characterised are retained (even if they are redundant).

We adivse running the visualisation of the sequence analysis with and without redundant sequences removed.

To remove redundant sequences from the dataset, set the `remove_redundant` parameter to `True` when calling `load_data()`.

In [ ]:
# Run this block before loading data

def load_data(data_file, fam):
    """Load the output data from BLASTP+/diamond into a pandas dataframe, and calculate the BLAST score ratio.
    
    :param data_file: str, path to the DIAMOND output file
    :param fam: str, name of CAZy family
    
    Return a pandas dataframe
    """
    df = pd.read_csv(data_file, sep='\t', header=None)
    
    column_names = ['qseqid', 'sseqid', 'qlen', 'slen', 'length', 'pident', 'evalue', 'bitscore']
    df.columns = column_names
    
    df['BSR'] = df['bitscore'] / df['qlen']
    
    df['qcov'] = df['length'] / df['qlen']
    df['scov'] = df['length'] / df['slen']
    
    df = remove_redunant_prots(df, fam)
    
    return df


def remove_redunant_prots(df, fam):
    """Identify groups of identical proteins, and retaining only one member per group
    
    :param df: pandas df containing data from DIAMOND
    :param fam: str, name of CAZy family - group name in the CANDIDATES df
    
    return df
    """

    # redundant proteins have a qcov of 1 and a pi of 100
    redundance_df = df.loc[ ( (df['pident'] == 100) & (df['qcov'] == 1) ) ]
    # find  the groups of redundant proteins
    redundant_grps = {}

    grp_num = 0

    parsed_prots = set()

    # identify groups of redundant proteins
    for ri in tqdm(range(len(redundance_df)), desc="Identifying IPGs"):
        row = redundance_df.iloc[ri]

        qseqid = row['qseqid']

        if qseqid in parsed_prots:
            continue  # has already been added

        # get all rows with the same query seq id
        qseqid_rows = redundance_df.loc[redundance_df['qseqid'] == qseqid]

        if len(qseqid_rows) == 1:
            continue  # aligned against self only

        subject_ids_to_add = set()

        # for each subject id
        # check if the versus is true, the qseqid is the sseqid when the sseqid is the qseqid
        for q_ri in range(len(qseqid_rows)):
            q_row = qseqid_rows.iloc[q_ri]
            sub_seqid = q_row['sseqid']

            # retrieve the row where the qseqid is now the subject, and the subject id is now the query seq
            # They are already in the redundancy df, therefore pident is 100 and qcov is 1
            sseqid_rows = redundance_df.loc[(
                (redundance_df['qseqid'] ==  sub_seqid) &
                (redundance_df['sseqid'] ==  qseqid))
            ]

            if len(sseqid_rows) > 0:
                subject_ids_to_add.add(sub_seqid)

        if len(subject_ids_to_add) > 0:
            # found redunant pairs for qseqid
            redundant_grps[grp_num] = {qseqid}

            for sub_seqid in subject_ids_to_add:
                redundant_grps[grp_num].add(sub_seqid)
                parsed_prots.add(sub_seqid)

            grp_num += 1

        parsed_prots.add(qseqid)

    # from each group select a representative protein
    # and identify members of the group that will be dropped
    removing = set()
    
    print(f"Identified {len(list(redundant_grps.keys()))} groups of identical proteins")

    for grp in redundant_grps:
        prots_to_keep = set{}
        
        for prot in redundant_grps[grp]:
            try:
                # retain proteins marked as candidates, functionally characitersed or structurally characterised
                if prot in CANDIDATES[fam]:
                    prots_to_keep.add(prot)
                elif prot in structured_prots[fam]:
                    prots_to_keep.add(prot)
                elif prot in functioned_prots[fam]:
                    prots_to_keep.add(prot)
                elif len(prots_to_keep) == 0: # ensure at least one protein from the group is retained
                    prots_to_keeps.add(prot)
                else:  # already have members from the group so drop the protein
                    removing.add(prot)
            except KeyError:
                if len(prots_to_keep) == 0:
                    prots_to_keeps.add(prot)
                else:  # already have members from the group so drop the protein
                    removing.add(prot)

    df = df[~df['qseqid'].isin(removing)]
    df = df[~df['sseqid'].isin(removing)]
    
    return df

## Visualise sequence diversity

This notebook generates a clustermap based upon a user defined variable. Heatmaps can also be generated to plot alternative variables, with proteins listed in the same order on the x and y-axis as the clustermap, to facilitate comparison of multiple variables.

We recommend:
* Generating a clustermap using the BLAST Score Ratio (BSR)
* Generating a heatmap of the percentage identiy with proteins listed in the same order as the clustermap
* Generating a heatmap of the query covereage with proteins listed in the same order as the clustermap

**Annotating candidates and functionally/structurally characterised proteins:**  
To generate a clustermap without annotating functionall/structurally characterised proteins, do not use the `annotate` parameter when calling `plot_clustermap()` and `plot_heatmap_of_clustermap()`. To annotate functionally/structurally characterised proteins set the `annotate` parameter to `True`.

**Plotting only candidates and functionally/structurally characterised proteins:**
To plot all proteins in a group (i.e. a CAZy family) do not call the `char_only` parameter when calling `plot_clustermap()` and `plot_heatmap_of_clustermap()`. To plot only the candidates and functionally/structurally characterised proteins, set the `char_only` parameter to `True`.

In [ ]:
# Run this block before visualising the data

def plot_clustermap(
    df,
    fam,
    varaible,
    title=None,
    colour_scheme='rocket_r',
    fig_size=(25, 25),
    save_fig=None,
    dpi=100,
    annotate=False,
    char_only=False,
):
    """Plot a cluster map for the specified variable
    
    :param df: pandas dataframe
    :param fam: str, CAZy family of interest
    :param variable: df, name of column containing the variable to plot
    :param title: str, default none. Title of plot
    :param colour_scheme: str, default rocket_r, seaborn colour scheme for plot
    :param fig_size: tuple, len 2, default (25, 10)
    :param save_fig: str, path to save file, default none, don't save fig
    :param dpi: int, default 100, resolution of saved file image
    :param annotate: bool, add annotation of protein candidates, and functionally/structurally 
        characteirsed proteins
    :param char_only: bool, if set to true, only plot proteins labelled as candidates or
        functionally/structurally characteirsed proteins
    
    Return seaborn plot
    """
    df = df[['qseqid', 'sseqid', varaible]]
    
    if char_only:  # plot only proteins that are candidates and functionally/structurally characteirsed proteins
        charactised_prots = functioned_prots[fam] + structured_prots[fam] + CANDIDATES[fam]
        df = df[df['qseqid'].isin(charactised_prots)]
        df = df[df['sseqid'].isin(charactised_prots)]
    
    heatmap_data = pd.pivot_table(df, index='qseqid', columns='sseqid', values=varaible)
    heatmap_data.columns = list(heatmap_data.columns)
    heatmap_data.index = list(heatmap_data.columns)
    heatmap_data = heatmap_data.fillna(0)
    
    if annotate:
        # add extra info on structural and functional characterisation of the family
        extra_data = []

        for prot in list(heatmap_data.columns):
            if prot in CANDIDATES[fam]:
                if prot in functioned_prots[fam]:
                    extra_data.append(STRUCT_PAL_DICT['funcCand'])
                elif prot in structured_prots[fam]:
                    extra_data.append(STRUCT_PAL_DICT['structCand'])
                else:
                    extra_data.append(STRUCT_PAL_DICT['cand'])

            elif prot in structured_prots[fam]:
                extra_data.append(STRUCT_PAL_DICT['struct'])

            elif prot in functioned_prots[fam]:
                extra_data.append(STRUCT_PAL_DICT['func'])

            else:
                extra_data.append(STRUCT_PAL_DICT['nothing'])

        fig = sns.clustermap(
            heatmap_data,
            cmap=colour_scheme,
            figsize=fig_size,
            row_colors=extra_data,
            col_colors=extra_data,
        );

        # extra data legend
        for label in list(STRUCT_PAL_DICT.keys()):
            fig.ax_row_dendrogram.bar(0, 0, color=STRUCT_PAL_DICT[label], label=label, linewidth=0)

        l3 = fig.ax_row_dendrogram.legend(title='Characterisation', loc='upper right', ncol=1)
    
    else:
        fig = sns.clustermap(
            heatmap_data,
            cmap=colour_scheme,
            figsize=fig_size,
        );
    
    if save_fig is not None:
        fig.savefig(save_fig, dpi=dpi);
    
    return fig


def plot_heatmap_of_clustermap(
    fig,
    df,
    fam,
    varaible,
    title=None,
    colour_scheme='rocket_r',
    fig_size=(25, 25),
    save_fig=None,
    dpi=100,
    annotate=False,
    char_only=False,
):
    """Generate a heatmap for the defined variable, with proteins plotted in the same order as the provided
    clustermap (fig)
    
    :param fig: seaborn clustergrid of entire family, default None, clustermap,
    :param df: pandas dataframe
    :param fam: str, CAZy family of interest
    :param variable: df, name of column containing the variable to plot
    :param title: str, default none. Title of plot
    :param colour_scheme: str, default rocket_r, seaborn colour scheme for plot
    :param fig_size: tuple, len 2, default (25, 10)
    :param save_fig: str, path to save file, default none, don't save fig
    :param dpi: int, default 100, resolution of saved file image
    :param annotate: bool, add annotation of protein candidates, and functionally/structurally 
        characteirsed proteins
    :param char_only: bool, if set to true, only plot proteins labelled as candidates or
        functionally/structurally characteirsed proteins
    
    Return nothing
    """
    column_order = list(fig.__dict__['data2d'].keys())
    row_order = list(fig.__dict__['data2d'].index)
    
    df = df[['qseqid', 'sseqid', varaible]]
    
    if char_only:  # plot only proteins that are candidates and functionally/structurally characteirsed proteins
        charactised_prots = functioned_prots[fam] + structured_prots[fam] + CANDIDATES[fam]
        df = df[df['qseqid'].isin(charactised_prots)]
        df = df[df['sseqid'].isin(charactised_prots)]
    
    heatmap_data = pd.pivot_table(df, index='qseqid', columns='sseqid', values=varaible)
    heatmap_data.columns = list(heatmap_data.columns)
    heatmap_data.index = list(heatmap_data.columns)
    heatmap_data = heatmap_data.fillna(0)
    
    heatmap_data = heatmap_data.to_dict()  # {col: {row: value}}

    heatmap_df_data = {}

    for _prot in column_order:
        column_data = heatmap_data[_prot] # dict of {row: value} for the column
        
        for __prot in row_order:
            row_value = column_data[__prot]

            try:
                heatmap_df_data[_prot]  # column
            except KeyError:
                heatmap_df_data[_prot] = {}

            heatmap_df_data[_prot][__prot] = row_value
            
    if annotate:
        # add extra info on structural and functional characterisation of the family
        extra_data_col = []

        for prot in column_order:
            # candidate 1, funct candidate 0.75, structured 0.5, functional 0.25, nothing 0
            if prot in CANDIDATES[fam]:
                if prot in functioned_prots[fam]:
                    extra_data_col.append(STRUCT_PAL_DICT['funcCand'])
                elif prot in structured_prots[fam]:
                    extra_data_col.append(STRUCT_PAL_DICT['structCand'])
                else:
                    extra_data_col.append(STRUCT_PAL_DICT['cand'])

            elif prot in structured_prots[fam]:
                extra_data_col.append(STRUCT_PAL_DICT['struct'])

            elif prot in functioned_prots[fam]:
                extra_data_col.append(STRUCT_PAL_DICT['func'])

            else:
                extra_data_col.append(STRUCT_PAL_DICT['nothing'])

        extra_data_row = []

        for prot in row_order:
            # candidate 1, funct candidate 0.75, structured 0.5, functional 0.25, nothing 0
            if prot in CANDIDATES[fam]:
                if prot in functioned_prots[fam]:
                    extra_data_row.append(STRUCT_PAL_DICT['funcCand'])
                elif prot in structured_prots[fam]:
                    extra_data_row.append(STRUCT_PAL_DICT['structCand'])
                else:
                    extra_data_row.append(STRUCT_PAL_DICT['cand'])

            elif prot in structured_prots[fam]:
                extra_data_row.append(STRUCT_PAL_DICT['struct'])

            elif prot in functioned_prots[fam]:
                extra_data_row.append(STRUCT_PAL_DICT['func'])

            else:
                extra_data_row.append(STRUCT_PAL_DICT['nothing'])

        fig = sns.clustermap(
            heatmap_df_data,
            cmap=colour_scheme,
            figsize=fig_size,
            row_cluster=False,
            col_cluster=False,
            row_colors=extra_data_row,
            col_colors=extra_data_col,
        );            

        # extra data legend
        for label in list(STRUCT_PAL_DICT.keys()):
            fig.ax_row_dendrogram.bar(0, 0, color=STRUCT_PAL_DICT[label], label=label, linewidth=0)

        l3 = fig.ax_row_dendrogram.legend(title='Info', loc='upper right', ncol=1)
    
    else:
        fig = sns.clustermap(
            heatmap_df_data,
            cmap=colour_scheme,
            figsize=fig_size,
            row_cluster=False,
            col_cluster=False,
        );

    if save_fig is not None:
        fig.savefig(save_fig, dpi=dpi);
    
    fig

## Family analysis

Here is some example code for running the analysis for CAZy family PL20.


In [ ]:
pl20_df = load_diamond_data('pl20_blastp_out', 'PL20')

In [ ]:
pl20_bsr_plt = plot_clustermap(pl20_df, 'PL20', 'BSR', fig_size=(100, 100), save_fig='pl20_clustermap.png')
pl20_bsr_plt

In [ ]:
# plot a clustermap of only the candidates and functionally/structurally characterised proteins
# that is also annotated to differentiate, candidates and functionally/structurally characterised proteins
pl20_char_bsr_plt = plot_clustermap(
    pl4_df,
    'PL20',
    'BSR',
    fig_size=(7, 7),
    save_fig='pl20_clustermap_char.png',
    char_only=True,
    annotate=True,
)
pl20_char_bsr_plt

Then plot the precentage identity and query coverage for the candidate and functionally/structurally characterised proteins, plotting the proteins on the heatmaps in the same order as they appear in the clustermap.

In [ ]:
print('PL20 percentage identity, colour scheme blue')
plot_heatmap_of_clustermap(
    pl20_char_bsr_plt,
    pl20_df,
    'PL20',
    'pident',
    fig_size=(7, 7),
    save_fig='pl20_clustermap_PIDENT_char.png',
    colour_scheme=sns.color_palette("Blues", as_cmap=True),
)

In [ ]:
print('PL20 query coverage, colour scheme purple')
plot_heatmap_of_clustermap(
    pl20_char_bsr_plt,
    pl20_df,
    'PL20',
    'qcov',
    fig_size=(7, 7),
    save_fig='pl20_clustermap_QCOV_char.png',
    colour_scheme=sns.color_palette("Purples", as_cmap=True),
)